# Binary Search Algorithms
###### as presented in "Algorithm Design with Haskell"

Binary search is a kind of divide and conquer algorithm where each recursive step is half the size of it's caller.

Given a monotonic function, say we want to find an input that produces a certain value.

In [2]:
search :: (Int -> Int) -> Int -> [Int]
search f t = [ x | x <- [0..t], t == f x ]

This spec uses linear search to find the answer but it's far better to do a binary search

In [3]:
search :: (Int -> Int) -> Int -> [Int]
search f t = seek (0, t)
  where
    seek (a, b) | a > b = []
                | t < fm = seek (a, m - 1)
                | t == fm = [m]
                | otherwise = seek (a, m + 1)
                where
                  m = (a + b) `div` 2
                  fm = f m

The main problem here is that we are searching over far to great a bounds. For some functions, this could produce values that overflow `Int` and produce incorrect results. Not to mention the expense of calculation. There are also worst case three comparisons performed at each step, which we will also improve upon.

Rather than searching over the full [0..t] range, we can find a bounded range (a, b) such that `f a < t <= f b`. Then we can search for t in the range `[a+1 .. b]`

In [15]:
bound :: (Int -> Int) -> Int -> (Int, Int)
bound f t = if t <= f 0
            then (-1, 0)
            else (b `div` 2, b)
  where
    b = until done (* 2) 1
    done x = t <= f x
    
search :: (Int -> Int) -> Int -> [Int]
search f t
  | f x == t = [x]
  | otherwise = []
  where
    x = smallest (bound f t) f t
    
smallest (a, b) f t
  | a + 1 == b = b
  | t <= f m = smallest (a, m) f t
  | otherwise = smallest (m, b) f t
  where m = (a + b) `div` 2

### Two-dimensional Search

Now instead of searching a simple range, we will search a grid. Given a monotonic function which takes two arguments, find all inputs that generate a result equal to a given value.

In [10]:
search :: (Int -> Int -> Int) -> Int -> [(Int, Int)]
search f t = [ (x, y) | x <- [0..t], y <- [0..t], f x y == t ]

This starts at the bottom left corner of the grid and does a linear search of both axis.

Instead we will start in the top left corner and do what is known as saddle back search.

In [18]:
:ext BangPatterns

search :: (Int -> Int -> Int) -> Int -> [(Int, Int)]
search f t = searchIn (0, p)
  where
  searchIn (!x, !y)
    | x > q || y < 0 = []
    | z < t = searchIn (x + 1, y)
    | z == t = (x, y) : searchIn (x + 1, y - 1)
    | otherwise = searchIn (x, y - 1)
    where z = f x y
  p = smallest (-1, t) (\y -> f 0 y) t
  q = smallest (-1, t) (\x -> f x 0) t

We can also use a divide conquer approach where we first test the center element (like the fulcrum when doing binary search) and eliminate some quandrants based on that.